In [1]:
import cv2
import os
import numpy as np
import hashlib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

def derive_key(password):
    """Derive a 32-byte AES key from the password using SHA-256."""
    return hashlib.sha256(password.encode()).digest()

def encrypt_message_aes(message, password):
    """Encrypt the message using AES with CBC mode."""
    key = derive_key(password)
    iv = os.urandom(16)  # Generate a random IV

    # Pad the message to a multiple of 16 bytes
    padder = PKCS7(128).padder()
    padded_message = padder.update(message.encode()) + padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(padded_message) + encryptor.finalize()

    return iv + encrypted_message  # Store IV with encrypted data

def decrypt_message_aes(encrypted_message, password):
    """Decrypt the message using AES with CBC mode."""
    key = derive_key(password)
    iv = encrypted_message[:16]  # Extract IV
    encrypted_data = encrypted_message[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    decrypted_padded = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad the message
    unpadder = PKCS7(128).unpadder()
    decrypted_message = unpadder.update(decrypted_padded) + unpadder.finalize()

    return decrypted_message.decode()

def encode_message(img_path, output_path, msg, password):
    img = cv2.imread(img_path)

    if img is None:
        print("Error: Image not found!")
        return
    
    height, width, _ = img.shape
    max_bytes = height * width * 3 - 18  # Reserve 18 bytes for metadata

    encrypted_msg = encrypt_message_aes(msg, password)
    
    if len(encrypted_msg) > max_bytes:
        print(f"Message too long! Max length is {max_bytes} bytes.")
        return

    msg_length = len(encrypted_msg)

    # Store message length in first pixel (3 bytes)
    img[0, 0, 0] = int(msg_length // (256 ** 2))  # Convert to integer
    img[0, 0, 1] = int((msg_length // 256) % 256)
    img[0, 0, 2] = int(msg_length % 256)

    # Store encrypted message in pixels
    n, m = 0, 1  # Start at second pixel
    index = 0
    for byte in encrypted_msg:
        img[n, m, index % 3] = int(byte)  # Convert bytes to integers
        if index % 3 == 2:  # Move to the next pixel after filling R, G, B
            m += 1
            if m >= width:
                n += 1
                m = 0
        index += 1

    cv2.imwrite(output_path, img)  # Save as PNG to prevent compression loss
    print("Message encrypted successfully!")
    os.system(f"start {output_path}")

def decode_message(img_path, password):
    img = cv2.imread(img_path)

    if img is None:
        print("Error: Image not found!")
        return
    
    #  Fix: Convert pixel values to Python integers before math operations
    length = (int(img[0, 0, 0]) * 256 * 256) + (int(img[0, 0, 1]) * 256) + int(img[0, 0, 2])

    encrypted_message = bytearray()
    n, m = 0, 1  # Start at second pixel
    index = 0
    for _ in range(length):
        encrypted_message.append(int(img[n, m, index % 3]))  # Convert to integer
        if index % 3 == 2:  # Move to the next pixel after reading R, G, B
            m += 1
            if m >= img.shape[1]:
                n += 1
                m = 0
        index += 1

    try:
        decrypted_msg = decrypt_message_aes(bytes(encrypted_message), password)
        print("Decrypted message:", decrypted_msg)
    except Exception:
        print("Decryption failed: Incorrect password or corrupted data.")

# Example Usage
img_file = "mypic.png"
output_file = "Encryptedmsg.png"

secret_msg = input("Enter secret message: ")
password = input("Enter password: ")

encode_message(img_file, output_file, secret_msg, password)

# Decryption
user_pass = input("Enter passcode for Decryption: ")
decode_message(output_file, user_pass)


Enter secret message:  Hi My Name is Tejas
Enter password:  54321


Message encrypted successfully!


Enter passcode for Decryption:  54321


Decrypted message: Hi My Name is Tejas
